In [156]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
import pandas as pn

In [157]:
# Скачали
data = pn.read_csv('salary-train.csv')

In [158]:
# Привели к нужному виду
data["FullDescription"] = data["FullDescription"].str.lower()
data["LocationNormalized"] = data["LocationNormalized"].str.lower()
data["ContractTime"] = data["ContractTime"].str.lower()
data = data.replace(to_replace='[^a-zA-Z0-9]', value=' ', regex=True)
data = data.replace(to_replace=' {2,}', value=' ', regex=True)
data.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k k uncappe...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury retail...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young ad...,sutton coldfield,NaN,20355


In [159]:
# Ловеркейс для нанов сделали заменой
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

In [160]:
# TFIDF для колонки с текстом
n = data["FullDescription"]
vector = TfidfVectorizer(min_df=5)
m = vector.fit_transform(n)

In [161]:
# Сделали несколько дамми-переменных из номинальных
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [162]:
from scipy.sparse import hstack

In [163]:
# Объединяем все на свете
features_train = hstack([m, X_train_categ])
target = data["SalaryNormalized"]

In [164]:
# Обучаем линейную модель
clf = Ridge(alpha=1.0, random_state=241)
n = clf.fit(features_train, target)

In [165]:
type(n)

sklearn.linear_model.ridge.Ridge

In [166]:
#Готовим тестовую выборку
data_test = pn.read_csv('salary-test-mini.csv')

In [167]:
data_test["FullDescription"] = data_test["FullDescription"].str.lower()
data_test["LocationNormalized"] = data_test["LocationNormalized"].str.lower()
data_test["ContractTime"] = data_test["ContractTime"].str.lower()
data_test = data_test.replace(to_replace='[^a-zA-Z0-9]', value=' ', regex=True)
data_test = data_test.replace(to_replace=' {2,}', value=' ', regex=True)
data_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


In [168]:
n = data["FullDescription"]
f = data_test["FullDescription"]
vector = TfidfVectorizer(min_df=5)
m = vector.fit_transform(n)
s = vector.transform(f)

In [169]:
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [170]:
features_test = hstack([s, X_test_categ])

In [173]:
# Ответ
g = clf.predict(features_test)
print(round(g[0], 2), round(g[1], 2))

56555.62 37188.32
